Per [R P Herrold's challenge](https://mail.python.org/pipermail/centraloh/2017-June/003124.html).

The hardest part was figuring out what the program
was supposed to accomplish.

From the email, code documentation, and code itself,
I had a very difficult time understanding what the code
was trying to accomplish, nevermind how the code was 
accomplishing it.

After studying the code, I made my description
of what the code is to accomplish. This was 
corroborated by the example output.

    for number of months from 0 to 11 inclusive,
    calculate the maximum possible number of days
    in (number of months + 1) consecutive months
    have. (Choose the starting month that gives highest possible answer.)
    in starting_month and the next
    
n_months months for all possible starting_months

Later on, I figured out the first column in the sample
output was the earliest or latest starting month in a year
that would produce the max number of days for n months.

For all my code below, months are numbered starting at 0.
That is,

0 means January

11 means December

In [1]:
MONTHS_PER_YEAR = 12

In [2]:
# for unknown year
def max_month_length(month):
    """Return maximum number of days for given month.
    month is zero-based.
    That is,
    0 means January,
    11 means December,
    12 means January (again)
    -2 means November (yup, wraps around both ways)"""
    max_month_lengths = (
        31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31)
    return max_month_lengths[month % len(max_month_lengths)]

In [3]:
max_month_length(0)  # January

31

In [4]:
max_month_length(13)  # February

29

In [5]:
max_month_length(-2)  # November

30

In [6]:
def max_days_n_months(n_months, starting_month=None):
    """Return the maximum number of days
    in n_months whole consecutive months,
    optionally starting only with starting_month.
    
    If starting_month is not specified,
    return highest value for all possible starting months."""
    if starting_month is not None:
        return sum(
            max_month_length(month)
            for month in range(starting_month, starting_month + n_months)
        )
    
    return max(
        max_days_n_months(n_months, starting_month)
        for starting_month in range(MONTHS_PER_YEAR)
    )

In [7]:
def foo(n):
    for n_months in range(1, n+1):
        n_days = max_days_n_months(n_months)
        yield n_months, n_days

In [8]:
n = MONTHS_PER_YEAR
# %timeit list(foo(n))
list(foo(n))

[(1, 31),
 (2, 62),
 (3, 92),
 (4, 123),
 (5, 153),
 (6, 184),
 (7, 215),
 (8, 245),
 (9, 276),
 (10, 306),
 (11, 337),
 (12, 366)]

The above output has the correct maximum number of days.

Now to add the stuff that keeps track of which starting months can yield those 

In [9]:
from collections import defaultdict

def max_n_days_for_months():
    """Yield tuples of
        number of consecutive months,
        maximum number of days of those consecutive months
        which starting months produce that above maximum
    for all number of consecutive months up to a year."""
    for n_months in range(1, MONTHS_PER_YEAR+1):
        d = defaultdict(list)
        for starting_month in range(MONTHS_PER_YEAR):
            n_days = max_days_n_months(n_months, starting_month)
            d[n_days].append(starting_month)
        max_n_days = max(d)
        yield n_months, max_n_days, sorted(d[max_n_days])

In [10]:
# %timeit list(max_n_days_for_months())
list(max_n_days_for_months())

[(1, 31, [0, 2, 4, 6, 7, 9, 11]),
 (2, 62, [6, 11]),
 (3, 92, [2, 4, 5, 6, 7, 9, 10]),
 (4, 123, [4, 6, 9]),
 (5, 153, [2, 3, 4, 5, 6, 7, 8]),
 (6, 184, [2, 4, 6, 7]),
 (7, 215, [6]),
 (8, 245, [2, 4, 5]),
 (9, 276, [4]),
 (10, 306, [2, 3]),
 (11, 337, [2]),
 (12, 366, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])]

In [11]:
def pretty():
    for selector, name in ((min, '-gt'), (max, '-ge')):
        yield f'Compare: {name} '
        for n_months, max_n_days, months in max_n_days_for_months():
            month = selector(months)
            if n_months == 1:
                yield f'month: {month} month spans: {max_n_days} days  '
            elif n_months == 2:
                yield f'month: {month} plus following month spans: {max_n_days} days  '
            else:
                yield f'month: {month} plus following {n_months} months spans: {max_n_days} days  '
        yield ''

In [12]:
for line in pretty():
    print(line)

Compare: -gt 
month: 0 month spans: 31 days  
month: 6 plus following month spans: 62 days  
month: 2 plus following 3 months spans: 92 days  
month: 4 plus following 4 months spans: 123 days  
month: 2 plus following 5 months spans: 153 days  
month: 2 plus following 6 months spans: 184 days  
month: 6 plus following 7 months spans: 215 days  
month: 2 plus following 8 months spans: 245 days  
month: 4 plus following 9 months spans: 276 days  
month: 2 plus following 10 months spans: 306 days  
month: 2 plus following 11 months spans: 337 days  
month: 0 plus following 12 months spans: 366 days  

Compare: -ge 
month: 11 month spans: 31 days  
month: 11 plus following month spans: 62 days  
month: 10 plus following 3 months spans: 92 days  
month: 9 plus following 4 months spans: 123 days  
month: 8 plus following 5 months spans: 153 days  
month: 7 plus following 6 months spans: 184 days  
month: 6 plus following 7 months spans: 215 days  
month: 5 plus following 8 months spans: 245 

The above output matches days_spanned.sh.stdout, but is ugly.
The code above is not as readable as I like.

Later, I polished below.

In [13]:
known_good_output = ''.join(f'{line}\n' for line in pretty())

In [14]:
def pretty():
    for selector, name in ((min, '-gt'), (max, '-ge')):
        yield f'Compare: {name} '
        for n_months, max_n_days, months in max_n_days_for_months():
            month = selector(months)
            if n_months == 1:
                duration_prose = f'month'
            elif n_months == 2:
                duration_prose = f'plus following month'
            else:
                duration_prose = f'plus following {n_months} months'
            yield f'month: {month} {duration_prose} spans: {max_n_days} days  '
        yield ''

In [15]:
assert known_good_output == ''.join(f'{line}\n' for line in pretty())

In [16]:
MONTH_NAMES = '''
    January February March April May June
    July August September October November December
'''.split()

MONTH_NAMES

['January',
 'February',
 'March',
 'April',
 'May',
 'June',
 'July',
 'August',
 'September',
 'October',
 'November',
 'December']

In [17]:
# derived from https://stackoverflow.com/questions/38981302/converting-a-list-into-comma-separated-string-with-and-before-the-last-item

def oxford_comma_join(items, join_word='and'):
    # print(f'items={items!r} join_word={join_word!r}')
    items = list(items)
    if not items:
        return ''
    elif len(items) == 1:
        return items[0]
    elif len(items) == 2:
        return f' {join_word} '.join(items)
    else:
        return ', '.join(items[:-1]) + f', {join_word} ' + items[-1]


In [18]:
test_data = (
    ((('',),), ''),
    ((('lonesome term',),), 'lonesome term'),
    ((('here', 'there'),), 'here and there'),
    ((('you', 'me', 'I'), 'or'), 'you, me, or I'),
    ((['here', 'there', 'everywhere'], 'or'), 'here, there, or everywhere'),
)

for args, known_good_output in test_data:
    # print(f'args={args!r}, k={known_good_output!r}, output={oxford_comma_join(*args)!r}')
    assert oxford_comma_join(*args) == known_good_output

In [19]:
import inflect

p = inflect.engine()

In [20]:
from textwrap import wrap

In [21]:
def pretty():
    for n_months, max_n_days, months in max_n_days_for_months():
        month_names = (MONTH_NAMES[month] for month in months)
        yield (
            f"{n_months} consecutive {p.plural('month', n_months)} "
            f'can have at most {max_n_days} days '
            f"if starting in {oxford_comma_join(month_names, 'or')}."
        )

In [22]:
for long_line in pretty():
    for line in wrap(long_line):
        print(line)

1 consecutive month can have at most 31 days if starting in January,
March, May, July, August, October, or December.
2 consecutive months can have at most 62 days if starting in July or
December.
3 consecutive months can have at most 92 days if starting in March,
May, June, July, August, October, or November.
4 consecutive months can have at most 123 days if starting in May,
July, or October.
5 consecutive months can have at most 153 days if starting in March,
April, May, June, July, August, or September.
6 consecutive months can have at most 184 days if starting in March,
May, July, or August.
7 consecutive months can have at most 215 days if starting in July.
8 consecutive months can have at most 245 days if starting in March,
May, or June.
9 consecutive months can have at most 276 days if starting in May.
10 consecutive months can have at most 306 days if starting in March
or April.
11 consecutive months can have at most 337 days if starting in March.
12 consecutive months can have 